In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!pip install scikit-dimension

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 680.9 kB/s eta 0:00:00


In [3]:
import sys
sys.path.append('/content/drive/MyDrive/Colab_Notebooks/HPPL/Final_project/HPPL-project-PHDim-main')
#import PHdim

In [3]:
import cupy as cp

In [2]:
import numpy as np
import skdim
#from scipy.spatial.distance import cdist
#from PHdim import PHD

In [80]:
from scipy.spatial.distance import cdist
import numpy as np
# import cupy as cp
from numba import jit
import multiprocessing as mp
import time



class PHD():

    def __init__(
        self,
        alpha=1.0,
        metric='euclidean',
        n_reruns=5,
        n_points=15,
        mst_method_name: str = 'classic'):
        '''
        Initializes the instance of PH-dim estimator
        Parameters:
        1) alpha --- real-valued parameter Alpha for computing PH-dim (see the reference paper). Alpha should be chosen
        lower than the ground-truth Intrinsic Dimensionality; however, Alpha=1.0 works just fine for our kind of data.
        2) metric --- String or Callable, distance function for the metric space (see documentation for Scipy.cdist)
        3) n_reruns --- Number of restarts of whole calculations
        4) n_points --- Number of subsamples to be drawn at each subsample
        '''
        self.alpha = alpha
        self.n_reruns = n_reruns
        self.n_points = n_points
        self.metric = metric
        self.is_fitted_ = False
        self.mst_method_name = mst_method_name

        method = {
            'classic': self.get_mst_value,
            'multiprocessing': self.get_mp_mst_value,
            'numba': self.get_nb_mst_value,
            'cupy': self.get_cp_mst_value,
        }

        if mst_method_name not in method.keys():
            raise ValueError(f'Method {mst_method_name} is not implemented')
        self.mst_method = method[mst_method_name]

    def _generate_samples(self, dist_mat, min_points):
        n = dist_mat.shape[0]
        test_n = np.linspace(min_points, n * 0.9, self.n_points).astype(int)
        random_indices = []
        for i in np.repeat(test_n, self.n_reruns):
            random_indices.append(np.random.choice(n, size=i, replace=False))
        return random_indices, test_n

    def _prim_tree(self, adj_matrix, ids, alpha=1.0):

        adj_matrix = adj_matrix[np.ix_(ids,ids)]

        infty = np.max(adj_matrix) + 10

        dst = np.ones(adj_matrix.shape[0]) * infty
        visited = np.zeros(adj_matrix.shape[0], dtype=bool)
        ancestor = -np.ones(adj_matrix.shape[0], dtype=int)

        v, s = 0, 0.0
        for i in range(adj_matrix.shape[0] - 1):
            visited[v] = 1
            ancestor[dst > adj_matrix[v]] = v
            dst = np.minimum(dst, adj_matrix[v])
            dst[visited] = infty

            v = np.argmin(dst)
            s += (adj_matrix[v][ancestor[v]] ** alpha)

        return s.item()

    def _cp_prim_tree(self, adj_matrix, ids, alpha=1.0):

        adj_matrix = adj_matrix[cp.ix_(ids,ids)]

        infty = cp.max(adj_matrix) + 10

        dst = cp.ones(adj_matrix.shape[0]) * infty
        visited = cp.zeros(adj_matrix.shape[0], dtype=bool)
        ancestor = -cp.ones(adj_matrix.shape[0], dtype=int)

        v, s = 0, 0.0
        for i in range(adj_matrix.shape[0] - 1):
            visited[v] = 1
            ancestor[dst > adj_matrix[v]] = v
            dst = cp.minimum(dst, adj_matrix[v])
            dst[visited] = infty

            v = cp.argmin(dst)
            s += (adj_matrix[v][ancestor[v]] ** alpha)

        return s.item()


    def _mp_prim_tree(self, adj_matrix, ids, return_dict, l, alpha=1.0):

        adj_matrix = adj_matrix[cp.ix_(ids,ids)]

        infty = cp.max(adj_matrix) + 10

        dst = cp.ones(adj_matrix.shape[0]) * infty
        visited = cp.zeros(adj_matrix.shape[0], dtype=bool)
        ancestor = -cp.ones(adj_matrix.shape[0], dtype=int)

        v, s = 0, 0.0
        for i in range(adj_matrix.shape[0] - 1):
            visited[v] = 1
            ancestor[dst > adj_matrix[v]] = v
            dst = cp.minimum(dst, adj_matrix[v])
            dst[visited] = infty

            v = cp.argmin(dst)
            s += (adj_matrix[v][ancestor[v]] ** alpha)
        return_dict[l] = s.item()

    def get_mst_value(self, random_indices, dist_mat):
        mst_values = np.zeros(len(random_indices))
        for i, ids in enumerate(random_indices):
            mst_values[i] = self._prim_tree(dist_mat, ids)
        return mst_values

    def get_mp_mst_value(self, random_indices, dist_mat):

        mst_values = np.zeros(len(random_indices))
        # num_workers = mp.cpu_count()
        manager = mp.Manager()
        return_dict = manager.dict()
        pool = mp.Pool(2)
        for i, ids in enumerate(random_indices):
            pool.apply_async(self._mp_prim_tree, args = (dist_mat, ids, return_dict, i))
        pool.close()
        pool.join()

        for k in return_dict.keys():
            mst_values[k] = return_dict[k]

        return mst_values

    def get_cp_mst_value(self, random_indices, dist_mat):
        mst_values = cp.zeros(len(random_indices))
        for i, ids in enumerate(random_indices):
            mst_values[i] = self._cp_prim_tree(dist_mat, ids)
        return mst_values.get()

    def pairwise_distance_matrix(self, points):
        squared_distances = cp.sum(points**2, axis=1, keepdims=True) + cp.sum(points**2, axis=1) - 2 * cp.dot(points, points.T)
        distance_matrix = cp.sqrt(np.maximum(squared_distances, 0))
        return distance_matrix

    @jit
    def pairwise_distance_matrix_numba(self, points):
        n_points = points.shape[0]
        squared_distances = np.sum(points**2, axis=1, keepdims=True) + np.sum(points**2, axis=1) - 2 * np.dot(points, points.T)
        distance_matrix = np.sqrt(np.maximum(squared_distances, 0))

        return distance_matrix

    @jit
    def get_nb_mst_value(self, random_indices, dist_mat):
        mst_values = np.zeros(len(random_indices))
        for i, ids in enumerate(random_indices):
            mst_values[i] = self._prim_tree(dist_mat, ids)
        return mst_values


    def fit_transform(self, X, y=None, min_points = 50):
        '''
        Computing the PH-dim
        Parameters:
        1) X --- point cloud of shape (n_points, n_features),
        2) y --- fictional parameter to fit with Sklearn interface
        3) min_points --- size of minimal subsample to be drawn
        '''

        start_time_total = time.time()

        # Measure the time for cdist
        start_time_cdist = time.time()
        # def pairwise_distance_matrix(points):
        #      squared_distances = cp.sum(points**2, axis=1, keepdims=True) + cp.sum(points**2, axis=1) - 2 * cp.dot(points, points.T)
        #      distance_matrix = cp.sqrt(np.maximum(squared_distances, 0))
        #      return distance_matrix

        if self.mst_method_name == 'cupy':
          if self.metric == 'euclidean':
            dist_mat = self.pairwise_distance_matrix(X)
          else: print('Not implemented')

        if self.mst_method_name == 'numba':
          if self.metric == 'euclidean':
            dist_mat = self.pairwise_distance_matrix_numba(X)
          else: print('Not implemented')

        else:
            dist_mat = cdist(X, X, metric=self.metric)

        elapsed_time_cdist = time.time() - start_time_cdist
        print(f"Time taken by cdist: {elapsed_time_cdist} seconds")

        random_indices, x = self._generate_samples(dist_mat, min_points)

        # Measure the time for the loop
        start_time_loop = time.time()
        ##### HERE IS THE ONLY LOOP WE NEED TO SPEED UP #####
        mst_values = self.mst_method(random_indices, dist_mat)

        elapsed_time_loop = time.time() - start_time_loop
        print(f"Time taken by loop: {elapsed_time_loop} seconds")

        y = mst_values.reshape(-1, self.n_reruns).mean(axis = 1)

        x = np.log(x)
        y = np.log(y)
        N = self.n_points

        m = (N * (x * y).sum() - x.sum() * y.sum()) / (N * (x ** 2).sum() - x.sum() ** 2)

        # Record the end time for the entire algorithm
        end_time_total = time.time()
        total_time = end_time_total - start_time_total

        # Calculate the percentage of time spent in cdist relative to the total time
        percentage_time_cdist = (elapsed_time_cdist / total_time) * 100

        print(f"Total time for the algorithm: {total_time} seconds")
        print(f"Percentage time spent in cdist: {percentage_time_cdist:.2f}%")
        print(f"Percentage time spent in the loop: {(elapsed_time_loop / total_time) * 100:.2f}%")

        return 1 / (1 - m)

In [76]:
# creating data
X = np.zeros((10_000,512))
X[:,:5] = skdim.datasets.hyperBall(n = 10_000, d = 5, radius = 1, random_state = 0)


In [56]:
intD = PHD('classic').fit_transform(X)
print(intD)

Time taken by cdist: 34.118144273757935 seconds
Time taken by loop: 45.453988552093506 seconds
Total time for the algorithm: 79.58793807029724 seconds
Percentage time spent in cdist: 42.87%
Percentage time spent in the loop: 57.11%
4.756041872288001


In [57]:
intD = PHD(mst_method_name='cupy').fit_transform(cp.array(X))
print(intD)

Time taken by cdist: 0.004437685012817383 seconds
Time taken by loop: 282.55278038978577 seconds
Total time for the algorithm: 282.5722312927246 seconds
Percentage time spent in cdist: 0.00%
Percentage time spent in the loop: 99.99%
4.673447684092005


In [81]:
intD = PHD(mst_method_name = 'numba').fit_transform(X)
print(intD)

Time taken by cdist: 4.389850854873657 seconds
Time taken by loop: 47.64099144935608 seconds
Total time for the algorithm: 52.04610800743103 seconds
Percentage time spent in cdist: 8.43%
Percentage time spent in the loop: 91.54%
4.758746799202225


In [40]:
intD = PHD('multiprocessing').fit_transform(X)
print(intD)

Time taken by cdist: 0.010948657989501953 seconds
Time taken by loop: 0.5531389713287354 seconds
Total time for the algorithm: 0.5695559978485107 seconds
Percentage time spent in cdist: 1.92%
Percentage time spent in the loop: 97.12%
4.626164013113161
